In [ ]:
import sys
import numpy as np
import pandas as pd
import importlib
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

from google.cloud import secretmanager
from datastory import datastory

sys.path.append('../lib')
import utils
import pandas_utils
import pesys_utils
importlib.reload(utils)
importlib.reload(pesys_utils)
importlib.reload(pandas_utils)

In [ ]:
utils.set_secrets_as_env(split_on=":", secret_name="projects/193123067890/secrets/vebjorn-rekkebo/versions/latest")

In [ ]:
con = pesys_utils.open_pen_connection()
df_perioder_registrert = pandas_utils.pandas_from_sql('../sql/trygdetidsperioder_saksbehandler.sql', con=con, lowercase=True)
df_perioder_frekvens = pandas_utils.pandas_from_sql('../sql/trygdetidsperioder_saksbehandler_frekvens.sql', con=con, lowercase=True)
con.close()

In [ ]:
df_perioder_registrert.head()

In [ ]:
current_year = datetime.now().year

df = df_perioder_registrert[df_perioder_registrert.land == 'Utland'].reset_index(drop=True)
df = df[df.aar != current_year]
df = df[df.aar != 2008]
df["snitt"] = df.antall_grunnlag / df.antall_personer

In [ ]:
fig_n_grunnlag = px.bar(df, "aar", "antall_grunnlag", labels={'antall_grunnlag':'Antall perioder', 'aar':'År'}, title='Antall manuelt registrerte utlandsperioder i pesys per år')
fig_n_grunnlag.show()

In [ ]:
fig_n_personer = px.bar(df, "aar", "antall_personer", labels={'antall_personer':'Antall personer', 'aar':'År'}, title='Antall personer med minst én manuelt registrert utlandsperiode i pesys per år')
fig_n_personer.show()

In [ ]:
fig_snitt_grunnlag = px.bar(df, "aar", "snitt", labels={'aar':'År', 'snitt':'Antall perioder'}, title='Gjennomsnittlig antall manuelt registrerte utlandsperioder per person')
fig_snitt_grunnlag.show()

In [ ]:
df_perioder_frekvens = df_perioder_frekvens[df_perioder_frekvens.aar != current_year]
df_perioder_frekvens_utland = df_perioder_frekvens[df_perioder_frekvens.land == 'Utland']

df2 = group_df_on_var(df_perioder_frekvens_utland, 'aar')

In [ ]:
df2.tail()

In [ ]:
cutoffs = [2, 5, 10, 20]
fig4 = go.Figure()
fig4.update_layout(title="Utvikling i andel personer med x antall trygdetidsperioder i utlandet (av alle som har minst én)")
for cutoff in cutoffs:
    df3 = df2[df2.antall_grunnlag >= cutoff].groupby(['aar'], as_index=False).andel.sum()
    fig4.add_traces(go.Scatter(x=df3.aar, y=df3.andel, name=f"Minst {cutoff}"))
    fig4.update_yaxes(tickformat='.0%')
    fig4.update_layout(xaxis_title="År", yaxis_title="Andel")
fig4.show()

In [ ]:
df2_2022 = df2[df2.aar == 2022]

In [ ]:
df2_2022[df2_2022.antall_grunnlag >= 30].frekvens.sum()

In [ ]:
fig_histogram = px.histogram(df2_2022[df2_2022.antall_grunnlag < 30], "antall_grunnlag", "frekvens", nbins=30, labels={'antall_grunnlag':'Antall perioder'}, title='Fordeling av antall utlandsperioder per person')
fig_histogram.update_layout(yaxis_title="Frekvens")
fig_histogram.show()

In [ ]:
ds = datastory.DataStory("Trygdetidsperioder i utlandet")
ds.header("Trygdetidsperioder i utlandet")
ds.markdown("""Saksbehandlere opplever at det er veldig tidkrevende å saksbehandle utlandssaker. 
    Særlig er det veldig mye jobb å registrere trygdetidsperioder for personer med mange perioder i utlandet.
    Dette er informasjon man har i EESSI-løsningen, men som ikke automatisk overføres til pesys, slik at saksbehandler må registrere hver periode manuelt.
    Her ser vi på omfanget av personer med mange trygdetidsperioder i pesys de siste årene. 
    Dette er altså på tvers av alle ytelser i pesys, med alderspensjon og uføretrygd som de to klart største.
""")
ds.plotly(fig_n_personer.to_json())
ds.markdown("""Figuren over viser antall personer som har minst én manuelt utfylt trygdetidsperiode for hvert år. 
    Her ser vi en svært tydelig økning i personer med utlandske perioder fram til 2014, men mer stabilt etterpå.
""")
ds.plotly(fig_n_grunnlag.to_json())
ds.markdown("""Denne figuren viser det totale antallet trygdetidsperioder som er registrert av saksbehandler hvert år. 
    Her er det en oppgang også etter 2014, hvilket betyr at antallet perioder per person har økt som vi ser i neste figur.
    Denne figuren illustrerer best den totale arbeidsmengden for saksbehandlerne gjennom et år.
""")
ds.plotly(fig_snitt_grunnlag.to_json())
ds.markdown("""Her vises gjennomsnittlig antall utlandsperioder per person med minst én periode. Her må man være klar over at det er en mindre mengde personer med svært mange perioder som drar opp gjennomsnittet.
    Til sammenligning er median 2.
""")
ds.plotly(fig4.to_json())
ds.markdown("""For å tallfeste hvor ofte en saksbehandler må registrere *for* mange trygdetidsperioder ser vi på hvor mange av personene som har mer enn x perioder. 
    Hvis vi setter en imaginær grense på mindre enn 5 perioder per person, så er det 27% av personene som er over denne grensa.
""")
ds.plotly(fig_histogram.to_json())
ds.markdown("""Siste figuren viser fordelingen av antall perioder per person i 2022. Merk at x-aksen er kutta på 30 perioder for små grupperinger. Det var i underkant av 200 personer med mer enn 30 perioder.
""")
ds.publish(url="https://datamarkedsplassen.intern.nav.no/api")